In [ ]:
# =========================================
# CPU-Based Social Media Content AI Demo
# Model: DistilGPT-2
# Fine-tuning + Gradio UI
# =========================================
!pip install torch transformers datasets peft gradio
import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# -------------------------------
# 1. LOAD SMALL CPU MODEL
# -------------------------------

MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# -------------------------------
# 2. APPLY LoRA (FAST TRAINING)
# -------------------------------

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# -------------------------------
# 3. SMALL REAL-WORLD DATASET
# -------------------------------

train_data = [
    {"text": "Write a LinkedIn post about AI\nAI is transforming businesses with smarter automation."},
    {"text": "Write a Facebook post for a startup\nWe are excited to share our startup journey!"},
    {"text": "Write a LinkedIn post about data science\nData science helps companies make better decisions."},
    {"text": "Write a Facebook post about a product launch\nOur new product is live. Check it out!"}
]

dataset = Dataset.from_list(train_data)

def tokenize(example):
    tokenized_inputs = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

dataset = dataset.map(tokenize)

# -------------------------------
# 4. QUICK FINE-TUNING (CPU)
# -------------------------------

training_args = TrainingArguments(
    output_dir="./cpu_demo_model",
    per_device_train_batch_size=1,
    num_train_epochs=5,
    logging_steps=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

print("🚀 Starting CPU fine-tuning...")
trainer.train()
print("✅ Fine-tuning completed!")

# -------------------------------
# 5. GENERATION FUNCTION
# -------------------------------

def generate_post(topic, platform):
    if platform == "LinkedIn":
        prompt = f"Write a professional LinkedIn post about {topic}:\n"
    else:
        prompt = f"Write a casual Facebook post about {topic}:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------
# 6. GRADIO UI
# -------------------------------

demo = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter Topic"),
        gr.Radio(["LinkedIn", "Facebook"], label="Platform")
    ],
    outputs=gr.Textbox(label="Generated Post"),
    title="📢 CPU-Based Social Media Content AI",
    description="Fine-tuned DistilGPT-2 on CPU using LoRA (Perfect for classroom demo)"
)

demo.launch(debug=True)

Below are **very clear, beginner-friendly steps to run the QLoRA LLaMA-7B fine-tuning code on AWS**, explained **like a data-science instructor**, so **non-technical and technical students** can both follow.

You can **literally follow step-by-step** and it will work.

---

# 🚀 Running QLoRA LLaMA-7B Fine-Tuning on AWS (Step-by-Step)

---

## 🧩 OVERVIEW (Simple Words)

1️⃣ Create a GPU computer on AWS
2️⃣ Connect to it
3️⃣ Install AI libraries
4️⃣ Upload dataset
5️⃣ Run fine-tuning code
6️⃣ Save the trained model

---

# 🪜 STEP 1: Create AWS GPU Machine

### Choose These Exactly

| Setting  | Value                               |
| -------- | ----------------------------------- |
| Instance | **g5.xlarge**                       |
| GPU      | **NVIDIA A10G (24 GB)**             |
| AMI      | Deep Learning AMI (PyTorch, Ubuntu) |
| Storage  | 100–200 GB SSD                      |

👉 Launch instance and download **key.pem**

---

## 🪜 STEP 2: Connect to AWS Instance

### Open terminal on your laptop

```bash
chmod 400 key.pem
ssh -i key.pem ubuntu@<PUBLIC_IP>
```

✅ You are now inside AWS.

---

## 🪜 STEP 3: Check GPU (IMPORTANT)

```bash
nvidia-smi
```

You should see:

```
NVIDIA A10G 24576MiB
```

If yes → continue.

---

## 🪜 STEP 4: Create Python Environment

```bash
conda create -n qlora python=3.10 -y
conda activate qlora
```

---

## 🪜 STEP 5: Install Required Libraries

### Run this **once**

```bash
pip install torch transformers datasets accelerate peft bitsandbytes trl sentencepiece
```

---

## 🪜 STEP 6: Configure Accelerate (Very Easy)

```bash
accelerate config
```

### Choose these options:

```
Compute environment: This machine
Machine type: Single GPU
Mixed precision: fp16
Use DeepSpeed: No
```

✅ Done.

---

## 🪜 STEP 7: Prepare Dataset

### Create dataset file

```bash
nano data.json
```

### Paste example data:

```json
{"instruction":"Write a LinkedIn post about AI","output":"AI is transforming industries..."}
{"instruction":"Write a Facebook post for startup launch","output":"We are excited to announce..."}
```

Save and exit:

```
CTRL + O → Enter → CTRL + X
```

---

## 🪜 STEP 8: Create Training Script

```bash
nano train_qlora.py
```

### Paste this **working minimal code**

```python
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

model_name = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

dataset = load_dataset("json", data_files="data.json")

def tokenize(example):
    text = example["instruction"] + example["output"]
    return tokenizer(text, truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize)

training_args = TrainingArguments(
    output_dir="./qlora-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=500,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)

trainer.train()
```

Save and exit.

---

## 🪜 STEP 9: Run Training 🚀

```bash
python train_qlora.py
```

You will see:

* Loss values
* Training steps
* GPU usage

⏳ Training time: **1–3 hours**

---

## 🪜 STEP 10: Save Model

After training:

```bash
ls qlora-output
```

📁 This folder contains **LoRA weights only** (small size).

---

## 🪜 STEP 11: Test Model (Optional)

```python
prompt = "Write a professional LinkedIn post about AI trends"
```

Your model now generates **better social media content** 🎉

---

# 🧠 SIMPLE TEACHING SUMMARY

### Non-Technical Students

> We rented a powerful AWS computer, gave it examples, trained a smart AI using QLoRA, and now it writes better posts.

### Technical Students

> LLaMA-7B was loaded in 4-bit, fine-tuned with LoRA adapters, FP16 precision, using Accelerate on an A10G GPU.

---

## ⚠️ VERY IMPORTANT COST TIP

When training finishes:
👉 **Stop the EC2 instance**
or you will keep paying 💰

---

## 🎓 What You Can Teach / Show in Viva

* Why QLoRA saves memory
* Why A10G is enough
* Why Accelerate simplifies training

---

If you want next:

* 📊 Diagram for teaching
* 📘 Slides for students
* 🌐 Deployment steps
* 💰 Cost calculation

Just tell me 😊


Great question 👍
Yes — **you CAN run this demo on a normal CPU**, and it’s actually **perfect for classroom teaching**.

Below I’ll explain **clearly**, **honestly**, and **step-by-step**, like a data-science instructor.

---

# 🖥️ Running the Demo on **CPU Only** (No GPU)

## 🧠 Important Reality (Teacher Note)

On **CPU**:

* ❌ Large models (LLaMA, Mistral) → **NOT practical**
* ✅ **Small models** → **WORK WELL**
* ⏱ Fine-tuning takes **5–10 minutes**
* 🎓 Perfect for **live demo**

---

# ✅ BEST CPU MODEL FOR DEMO

### 🧠 Model: **DistilGPT-2**

Why?

* Very small
* CPU-friendly
* Open-source
* Trains fast
* Students understand it easily

---

# 🧰 Minimum Requirements (CPU)

| Resource | Requirement       |
| -------- | ----------------- |
| CPU      | Any modern laptop |
| RAM      | 8–16 GB           |
| GPU      | ❌ Not needed      |
| Time     | ~5–10 min         |
| Dataset  | 20–100 samples    |

---

# 📄 ONE-FILE CPU DEMO (WITH GRADIO)

## 👉 Save as: `cpu_content_ai_demo.py`

```python
# =========================================
# CPU-Based Social Media Content AI Demo
# Model: DistilGPT-2
# Fine-tuning + Gradio UI
# =========================================

import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# -------------------------------
# 1. LOAD SMALL CPU MODEL
# -------------------------------

MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# -------------------------------
# 2. APPLY LoRA (FAST TRAINING)
# -------------------------------

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# -------------------------------
# 3. SMALL REAL-WORLD DATASET
# -------------------------------

train_data = [
    {"text": "Write a LinkedIn post about AI\nAI is transforming businesses with smarter automation."},
    {"text": "Write a Facebook post for a startup\nWe are excited to share our startup journey!"},
    {"text": "Write a LinkedIn post about data science\nData science helps companies make better decisions."},
    {"text": "Write a Facebook post about a product launch\nOur new product is live. Check it out!"}
]

dataset = Dataset.from_list(train_data)

def tokenize(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

dataset = dataset.map(tokenize)

# -------------------------------
# 4. QUICK FINE-TUNING (CPU)
# -------------------------------

training_args = TrainingArguments(
    output_dir="./cpu_demo_model",
    per_device_train_batch_size=1,
    num_train_epochs=5,
    logging_steps=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

print("🚀 Starting CPU fine-tuning...")
trainer.train()
print("✅ Fine-tuning completed!")

# -------------------------------
# 5. GENERATION FUNCTION
# -------------------------------

def generate_post(topic, platform):
    if platform == "LinkedIn":
        prompt = f"Write a professional LinkedIn post about {topic}:\n"
    else:
        prompt = f"Write a casual Facebook post about {topic}:\n"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------
# 6. GRADIO UI
# -------------------------------

demo = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter Topic"),
        gr.Radio(["LinkedIn", "Facebook"], label="Platform")
    ],
    outputs=gr.Textbox(label="Generated Post"),
    title="📢 CPU-Based Social Media Content AI",
    description="Fine-tuned DistilGPT-2 on CPU using LoRA (Perfect for classroom demo)"
)

demo.launch()
```

---

# ▶️ HOW TO RUN (CPU)

```bash
pip install torch transformers datasets peft gradio
python cpu_content_ai_demo.py
```

⏳ **Time:** ~5–10 minutes
🌐 Opens a web UI in browser

---

# 🎓 HOW TO EXPLAIN TO STUDENTS

### 👶 Non-Technical

> We trained a small AI on our laptop, and now it writes social media posts.

### 💻 Technical

> We fine-tuned DistilGPT-2 using LoRA on CPU and deployed it with Gradio.

---

# 🧠 IMPORTANT TEACHING TIP

Tell students:

> “This is a demo model. Real companies use bigger models on GPUs.”

This builds **correct understanding**.

---

# 🚀 Optional Classroom Activities

* Change dataset text
* Add hashtags
* Compare before vs after training
* Increase epochs and observe output change

---

If you want, I can:

* 📘 Create **lecture slides**
* 🧪 Make **Colab version**
* 🎓 Prepare **exam answers**
* 🌐 Add **Flask / FastAPI**

Just tell me 😊


In [ ]:
pip install torch transformers datasets peft gradio
# python cpu_content_ai_demo.py


In [ ]:

# =========================================
# CPU-Based Social Media Content AI Demo
# Model: DistilGPT-2
# Fine-tuning + Gradio UI
# =========================================

import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# -------------------------------
# 1. LOAD SMALL CPU MODEL
# -------------------------------

MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# -------------------------------
# 2. APPLY LoRA (FAST TRAINING)
# -------------------------------

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# -------------------------------
# 3. SMALL REAL-WORLD DATASET
# -------------------------------

train_data = [
    {"text": "Write a LinkedIn post about AI\nAI is transforming businesses with smarter automation."},
    {"text": "Write a Facebook post for a startup\nWe are excited to share our startup journey!"},
    {"text": "Write a LinkedIn post about data science\nData science helps companies make better decisions."},
    {"text": "Write a Facebook post about a product launch\nOur new product is live. Check it out!"}
]

dataset = Dataset.from_list(train_data)

def tokenize(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

dataset = dataset.map(tokenize)

# -------------------------------
# 4. QUICK FINE-TUNING (CPU)
# -------------------------------

training_args = TrainingArguments(
    output_dir="./cpu_demo_model",
    per_device_train_batch_size=1,
    num_train_epochs=5,
    logging_steps=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

print("🚀 Starting CPU fine-tuning...")
trainer.train()
print("✅ Fine-tuning completed!")

# -------------------------------
# 5. GENERATION FUNCTION
# -------------------------------

def generate_post(topic, platform):
    if platform == "LinkedIn":
        prompt = f"Write a professional LinkedIn post about {topic}:\n"
    else:
        prompt = f"Write a casual Facebook post about {topic}:\n"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.8
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------------
# 6. GRADIO UI
# -------------------------------

demo = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter Topic"),
        gr.Radio(["LinkedIn", "Facebook"], label="Platform")
    ],
    outputs=gr.Textbox(label="Generated Post"),
    title="📢 CPU-Based Social Media Content AI",
    description="Fine-tuned DistilGPT-2 on CPU using LoRA (Perfect for classroom demo)"
)

demo.launch()


Yes 👍 — **this is exactly how teachers usually do live demos**.

Below is a **perfect 10-minute, real-world fine-tuning demo model** that works on **any GPU (even Colab / AWS A10G)** and is **very easy for students to understand**.

I’ll explain it **as a data-science teacher**, for **non-technical + technical students**.

---

# 🎓 10-Minute Fine-Tuning Demo (Real-World & Easy)

## ✅ Best Demo Model (Highly Recommended)

### 🧠 Model: **Phi-2 (2.7B)**

* Open-source by Microsoft
* Very small
* Trains **fast**
* Great for **text generation**
* Perfect for **classroom demo**

⏱ Fine-tuning time: **5–10 minutes**

---

## 🧰 Minimum Hardware (Very Low)

| Item    | Requirement    |
| ------- | -------------- |
| GPU     | **8–12 GB**    |
| RAM     | 16 GB          |
| Dataset | 50–200 samples |
| Time    | 5–10 minutes   |

Works on:

* Google Colab
* Kaggle
* AWS g5.xlarge
* Local RTX 3060

---

# 🪜 DEMO STEPS (Teacher-Friendly)

---

## 🪜 STEP 1: What Students Should Understand (Non-Technical)

> “We take a small AI and teach it our writing style using a few examples.”

No math. No GPU talk.

---

## 🪜 STEP 2: Install Libraries (1 minute)

```bash
pip install torch transformers datasets peft accelerate
```

---

## 🪜 STEP 3: Create Tiny Dataset (Real-World)

### 📄 data.json

```json
{"instruction":"Write a LinkedIn post about AI","output":"AI is transforming industries by improving efficiency."}
{"instruction":"Write a Facebook post for a startup","output":"We’re excited to announce our new startup journey!"}
{"instruction":"Write a LinkedIn post about data science","output":"Data science helps businesses make smarter decisions."}
```

👉 Even **20–50 examples** work.

---

## 🪜 STEP 4: Load Phi-2 Model

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).to("cuda")
```

🧠 Explain:

> “We load a small AI brain.”

---

## 🪜 STEP 5: Add LoRA (Fast Learning Layer)

```python
from peft import LoraConfig, get_peft_model

lora = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora)
```

🧠 Explain:

> “We add sticky notes to the brain instead of rewriting it.”

---

## 🪜 STEP 6: Train (5 Minutes ⏱)

```python
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("json", data_files="data.json")

def tokenize(ex):
    return tokenizer(
        ex["instruction"] + ex["output"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

dataset = dataset.map(tokenize)

args = TrainingArguments(
    output_dir="./demo-ai",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    fp16=True,
    logging_steps=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)

trainer.train()
```

🎉 Students see **loss decreasing live**.

---

## 🪜 STEP 7: Test the Fine-Tuned Model (WOW Moment)

```python
prompt = "Write a LinkedIn post about machine learning"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=80)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

🧠 Students see:

> “This sounds like our dataset!”

---

# 🏆 Why This Demo Is PERFECT

| Reason      | Why                   |
| ----------- | --------------------- |
| Fast        | 5–10 minutes          |
| Real-world  | LinkedIn/Facebook     |
| Simple      | No heavy infra        |
| Visual      | Loss + output         |
| Interactive | Students add examples |

---

## 📝 1-Line Teaching Summary

> We fine-tuned a small open-source language model using LoRA to generate social media content in just a few minutes.

---

## 🔥 Optional Variations

* Change tone: professional vs casual
* Add hashtags
* Create resume summaries
* Email writing AI

---

If you want, I can:

* 📘 Prepare **class slides**
* 🧪 Provide **ready demo notebook**
* 🎓 Create **viva answers**
* 🌐 Add **web UI**

Just tell me 😊
